### Importación de las librerías necesarias y url de la que parten los datos climatológicos

El objetivo de este Notebook es crear el dataframe de predicción con las variables financieras y climatológicas.

Las variables climatológicas salieron de la siguiente URL.

https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=fa8357cec5efa610VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

In [13]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

import os
directorios = os.listdir('Data/Climatología')

def read_climate_data(df, cod_magnitud, cod_estacion, magnitud):
    
    """Funcion que lee los datos climáticos de un archivo csv por cada año.
    Parametros:
        -df: dataframe que proviene de un archivo csv con los datos climatológicos agrupados
        -cod_magnitud: código de la magnitud que se quiera según la documentación de la página de la url anterior
        -cod_estacion: estación de la que se quiera sacar el dato
        -magnitud: nombre que queremos asociar a la serie temporal
    Output:
        - Serie temporal de la magnitud seleccionada
    """
    
    # Filtramos por la magnitud y la estación, ya que cada magnitud tiene
    df = df[(df.loc[:, "MAGNITUD"] == cod_magnitud) & (df.loc[:, "ESTACION"] == cod_estacion)]

    # Cogemos las columnas de las horas
    cols = ["H01", "H02", "H03", "H04", "H05", "H06", "H07", "H08",
            "H09", "H10", "H11", "H12", "H13", "H14", "H15", "H16",
            "H17", "H18", "H19", "H20", "H21", "H22", "H23", "H24"]

    dic_cols = {}
    for i in range(24):
        dic_cols[cols[i]] = i
    
    # Filtramos el dataframe por las columnas que queremos
    df = df[["ANO", "MES", "DIA"] + cols]

    # Realizamos un melt para convertir la tabla en una serie temporal con 
    df = pd.melt(df, id_vars=['ANO', 'MES', 'DIA'], value_vars=cols,
            var_name='HORA', value_name= magnitud)

    df.HORA = df.HORA.map(dic_cols)

    df.loc[:, "Fecha"] = [datetime(df.ANO[i], df.MES[i], df.DIA[i], df.HORA[i]) for i in range(len(df))]
    
    # Devolvermos en la función la serie temporal de la magnitud
    return df[["Fecha", magnitud]].set_index('Fecha')

# Creamos dataframes vacíos para ir incluyendo en ellos las distintas series temporales
temperatura_grouped, humedad_grouped, precipitaciones_grouped, radiacion_grouped, velocidad_grouped = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

for directorio in directorios:
    df = pd.read_csv('Data/Climatología/' + directorio, sep = ";")
    
    # Para cada magnitud cogemos su valor y le ponemos su nombre en la serie temporal
    temperaturas = read_climate_data(df, 83, 58, 'Temperatura')
    humedad_relativa = read_climate_data(df, 86, 58, 'Humedad_Relativa')
    precipitaciones = read_climate_data(df, 89, 102, 'Precipitacion')
    radiacion_solar = read_climate_data(df, 88, 102, 'Radiacion')
    velocidad_viento = read_climate_data(df, 81, 102, 'Velocidad_Viento')
    
    # Agregamos al dataframe vacío que se creó anteriormente cada iteración de cada fichero
    temperatura_grouped = pd.concat([temperatura_grouped, temperaturas])
    humedad_grouped = pd.concat([humedad_grouped, humedad_relativa])
    precipitaciones_grouped = pd.concat([precipitaciones_grouped, precipitaciones])
    radiacion_grouped = pd.concat([radiacion_grouped, radiacion_solar])
    velocidad_grouped = pd.concat([velocidad_grouped, velocidad_viento])
    
    # Ordenamos los índices
    temperatura_grouped = temperatura_grouped.sort_index()
    humedad_grouped = humedad_grouped.sort_index()
    precipitaciones_grouped = precipitaciones_grouped.sort_index()
    radiacion_grouped = radiacion_grouped.sort_index()
    velocidad_grouped = velocidad_grouped.sort_index()
    
# Hay atípicos en la temperatura en el 7 y 8 de septiembre de 2021 (todas las horas a -55 grados centígrados)
# Metemos los datos a mano porque son pocos

temperatura_grouped["Temperatura"][temperatura_grouped.loc[:, "Temperatura"] == -55] = [23.6,25.8,28.1,29. ,
            29.9,30.3,31.6,32. ,31.2,29.9,28.5,28. ,27.7,26.2, 23.3,22.2,21.5,20.8,21.1,20.2,19. ,18.2,18.1,19.4,23.6,25.8,28.1,29.]

# Creamos el dataframe climatológico. Tiene las mismas dimensiones así que podemos usar la función concat sin problema
df_climate =  pd.concat([temperatura_grouped,
                         humedad_grouped,
                         precipitaciones_grouped,
                         radiacion_grouped,
                         velocidad_grouped],
                        axis = 1)

# Creamos lasa columnas año, mes y dia para unirla luego al dataframe financiero
df_climate.loc[:, "ANO"] = df_climate.index.year
df_climate.loc[:, "MES"] = df_climate.index.month
df_climate.loc[:, "DIA"] = df_climate.index.day

### Creación de los mínimos, máximos y spreads

Al igual que se realizó con los precios

In [15]:
temperaturas_maximas = temperatura_grouped.groupby([temperatura_grouped.index.year,
                             temperatura_grouped.index.month,
                             temperatura_grouped.index.day]).max()
temperaturas_minimas = temperatura_grouped.groupby([temperatura_grouped.index.year,
                             temperatura_grouped.index.month,
                             temperatura_grouped.index.day]).min()

spread_temperaturas = temperaturas_maximas - temperaturas_minimas

temperaturas_maximas.index.names = [None, None, None]
temperaturas_maximas.columns = ['Temperatura_max']
temperaturas_minimas.index.names = [None, None, None]
temperaturas_minimas.columns = ['Temperatura_min']
spread_temperaturas.index.names = [None, None, None]
spread_temperaturas.columns = ['Spred_temperatura']

temperaturas_maximas = temperaturas_maximas.reset_index()
temperaturas_minimas = temperaturas_minimas.reset_index()
spread_temperaturas = spread_temperaturas.reset_index()

# Lectura de variables financieras

Se realizará con la api de Yahoo Finance para obtener el Brent el Api2 y el IBEX35

In [23]:
import yfinance as yf
import datetime 

def read_commodities(ticker, column):
    
    """Función que obtine las variables financieras de Yahoo finance:
    Parametros:
        -ticker: nombre del símbolo que hace referencia a la variable financiera
        -column: nombre que se le quiere poner a la serie temporal
    Output:
        - Serie temporal de la variable financiera seleccionada
    """
    # Vamos a obtener los valores desde 2019, ya que la variable de hueco térmico tiene esta antigüedad
    start = datetime.datetime(2019,1,1) 
    end = datetime.datetime(2022,1,4)
    
    # Accedemos a la api de yahoo finance
    df = yf.Ticker(ticker)
    
    # Nos quedamos con el precio de cierre
    df = df.history(start=start, end = end)[["Close"]]
    
    # Ponemos la serie con periodicidad diaria y rellenamos los nulos con el valor anterior (para fines de semana, se 
    # realiza así en el mercado real)
    df = df.asfreq('D', method = 'ffill')
    df.columns = [column]
    
    # Creamos las columnas año, mes y día para unirlas a las variables climatológicas
    df.loc[:, "ANO"] = df.index.year
    df.loc[:, "MES"] = df.index.month
    df.loc[:, "DIA"] = df.index.day
    return df
 
brent = read_commodities("BZ=F", 'Brent')
api2 = read_commodities("MTF=F", 'Api2')
ibex = read_commodities("^IBEX", 'IBEX')

### Unión del dataframe de clima con el dataframe financiero

In [25]:
df_climate = df_climate.reset_index().merge(temperaturas_maximas,
                 left_on = ['ANO', 'MES', 'DIA'],
                 right_on = ['level_0', 'level_1', 'level_2']) \
                .merge(temperaturas_minimas,
                 left_on = ['ANO', 'MES', 'DIA'],
                 right_on = ['level_0', 'level_1', 'level_2']) \
                .merge(spread_temperaturas,
                 left_on = ['ANO', 'MES', 'DIA'],
                 right_on = ['level_0', 'level_1', 'level_2']) \
                .merge(brent, on = ['ANO', 'MES', 'DIA']) \
                .merge(api2, on = ['ANO', 'MES', 'DIA']) \
                .merge(ibex, on = ['ANO', 'MES', 'DIA']) \
                .drop(columns = ['level_0_x', 'level_1_x', 'level_2_x',
                                'level_0_y', 'level_1_y','level_2_y',
                                'level_0', 'level_1', 'level_2',
                                'ANO','MES','DIA']) \
                .set_index('Fecha')

### Exportación de los datos procesados por este notebook

In [27]:
df_climate.to_csv('Data/datos_climatologicos.csv')